In [55]:
#타자 데이터
import urllib
from bs4 import BeautifulSoup
html       = urllib.urlopen('http://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx')
soup       = BeautifulSoup(html, "lxml")
table      = soup.find('table', attrs={'class':'tData01 tt'}) #class="tData01 tt"인 table태그를 찾음
table_body = table.find('tbody') #table 데이터 중 tbody태그를 찾음
rows       = table_body.find_all('tr') #table_tbody 데이터 중 tr태그를 전부 찾음
for row in rows:
    cells    = row.findAll('td') #각 td태그마다 저장된 데이터(text)들을 각 변수에 하나씩 저장
    Rank     = cells[0].find(text=True) #순위
    Name     = cells[1].find(text=True) #이름
    teamName = cells[2].find(text=True) #팀이름
    AVG      = cells[3].find(text=True) #타율
    G        = cells[4].find(text=True) #게임수
    PA       = cells[5].find(text=True) #타석
    AB       = cells[6].find(text=True) #타수
    R        = cells[7].find(text=True) #득점
    H        = cells[8].find(text=True) #안타
    _2B      = cells[9].find(text=True) #2루타
    _3B      = cells[10].find(text=True) #3루타
    HR       = cells[11].find(text=True) #홈런
    TB       = cells[12].find(text=True) #루타
    RBI      = cells[13].find(text=True) #타점
    SAC      = cells[14].find(text=True) #희생번트
    SF       = cells[15].find(text=True) #희생플라이
    print ("{0:4s}| {1:11s}| {2:4s}| {3:4s}| {4:4s}| {5:4s}| {6:4s}| {7:4s}| {8:4s}| {9:4s}| {10:4s}| {11:4s}| {12:4s}| {13:4s}| {14:4s}| {15:4s}"
           .format(Rank, Name.encode('utf-8'), teamName.encode('utf-8'), AVG, G, PA, AB, R, H, _2B, _3B, HR, TB, RBI, SAC, SF))


1   | 최형우  | 삼성| 0.376| 138 | 618 | 519 | 99  | 195 | 46  | 2   | 31  | 338 | 144 | 0   | 7   
2   | 김태균  | 한화| 0.365| 144 | 652 | 529 | 94  | 193 | 39  | 0   | 23  | 301 | 136 | 0   | 6   
3   | 이용규  | 한화| 0.352| 113 | 530 | 452 | 98  | 159 | 20  | 4   | 3   | 196 | 41  | 7   | 1   
4   | 김주찬  | KIA | 0.346| 130 | 555 | 511 | 97  | 177 | 37  | 3   | 23  | 289 | 101 | 1   | 6   
5   | 박용택  | LG  | 0.346| 138 | 578 | 509 | 84  | 176 | 24  | 0   | 11  | 233 | 90  | 0   | 7   
6   | 구자욱  | 삼성| 0.343| 108 | 495 | 428 | 105 | 147 | 19  | 13  | 14  | 234 | 77  | 2   | 5   
7   | 박민우  | NC  | 0.343| 121 | 515 | 435 | 84  | 149 | 16  | 6   | 3   | 186 | 55  | 10  | 7   
8   | 유한준  | kt  | 0.336| 110 | 460 | 408 | 70  | 137 | 22  | 0   | 14  | 201 | 64  | 0   | 3   
9   | 황재균  | 롯데| 0.335| 127 | 559 | 498 | 97  | 167 | 26  | 5   | 27  | 284 | 113 | 0   | 8   
10  | 박건우  | 두산| 0.335| 132 | 540 | 484 | 95  | 162 | 36  | 4   | 20  | 266 | 83  | 4   | 5   
11  | 고종욱  | 넥센| 0.334| 133 | 566 | 527 | 